In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [84]:
# Load the data files
#area_names = pd.read_csv("C:\\Users\\Legion\\OneDrive - UNIVERSITAS INDONESIA\\Documents\\capstone\\c241-rp03-bukitvista-room-pricing-ml\\data\\auxillary\\area_names.csv")
#property_designs = pd.read_csv("C:\\Users\\Legion\\OneDrive - UNIVERSITAS INDONESIA\\Documents\\capstone\\c241-rp03-bukitvista-room-pricing-ml\\data\\auxillary\\property_designs.csv")
#property_types = pd.read_csv("C:\\Users\\Legion\\OneDrive - UNIVERSITAS INDONESIA\\Documents\\capstone\\c241-rp03-bukitvista-room-pricing-ml\\data\\auxillary\\property_types.csv")
data = pd.read_csv("/content/CLEAN_RoomPrice_11_06_2024.csv")

## There will be several feature engineering

1. Date Features: Convert month numeric to string, Holiday Indicator, Convert to Datetime, Stay Duration, Booking Month, Day of Week, Booking Lead Time
2. Revenue Metrics: Total Earnings, Price Fluctuation,
3. Review Metrics: Total Review Score
4. Data Cleaning: Drop Unnecessary Columns

In [72]:
def feature_engineering(df, date_column, check_out_column, holiday_dates):
    # Ensure the date columns are datetime objects
    df[date_column] = pd.to_datetime(df[date_column], format='%Y-%m-%d')
    df[check_out_column] = pd.to_datetime(df[check_out_column], format='%Y-%m-%d')

    # Extract the month from the date column and convert it to a categorical variable
    df['month'] = df[date_column].dt.strftime('%B').str.lower()

    #holidays_df = pd.DataFrame(holiday_dates, columns=['date'])
    #df['contain_national_holiday'] = df[(holidays_df['date'] >= df[date_column]) & (holidays_df['date'] <= df[check_out_column])]

    # Check if the dates fall within any of the holiday ranges
    #for date in holiday_ranges:
    #    mask = (df[date_column] <= end_date) & (df[check_out_column] >= start_date)
    #    df.loc[mask, 'contain_national_holiday'] = True

    return df

In [70]:
def FE_ContainsHoliday(check_in_date, check_out_date, holiday_dates):
  check_in = pd.to_datetime(check_in_date, format='%d-%m-%Y')
  check_out = pd.to_datetime(check_out_date, format='%d-%m-%Y')

  for date in holiday_dates:
    if (date < check_out) & (date > check_in):
      return True
  return False

### Define holiday ranges (example range including Indonesia's Independence Day)

In [71]:
holiday_dates = [
        "01-01-2022", "01-02-2022", "28-02-2022", "03-03-2022", "15-04-2022", "01-05-2022",
    "02-05-2022", "03-05-2022", "16-05-2022", "26-05-2022", "01-06-2022", "09-07-2022",
    "30-07-2022", "17-08-2022", "08-10-2022", "25-12-2022",
    "01-01-2023", "22-01-2023", "23-01-2023", "18-02-2023", "22-03-2023", "23-03-2023",
    "07-04-2023", "19-04-2023", "20-04-2023", "21-04-2023", "22-04-2023", "23-04-2023",
    "24-04-2023", "25-04-2023", "01-05-2023", "18-05-2023", "01-06-2023", "02-06-2023",
    "04-06-2023", "28-06-2023", "29-06-2023", "30-06-2023", "19-07-2023", "17-08-2023",
    "28-09-2023", "25-12-2023", "26-12-2023",
    "01-01-2024", "08-02-2024", "10-02-2024", "11-03-2024", "29-03-2024", "31-03-2024",
    "10-04-2024", "11-04-2024", "01-05-2024", "09-05-2024", "23-05-2024", "01-06-2024",
    "17-06-2024", "07-07-2024", "17-08-2024", "16-09-2024", "25-12-2024",
    "09-02-2024", "12-03-2024", "08-04-2024", "09-04-2024", "12-04-2024", "15-04-2024",
    "10-05-2024", "24-05-2024", "18-06-2024", "26-12-2024"
]

holiday_ranges = []
for date in holiday_dates:
  holiday_ranges.append(pd.to_datetime(date, format='%d-%m-%Y'))

## Apply featture

In [85]:
data = feature_engineering(data, 'booking_check_in', 'booking_check_out', holiday_ranges)
data.head()


,Unnamed: 0,room_id,unit_id,booking_id,booking_check_in,created_at,booking_check_out,earnings_in_idr,earnings_per_day,average_baseline_price,rating,review_sentiment_score,communication,cleanliness,accuracy,month
0,13,397667.0,UN2501,07-05-2023MartaRudy,2023-05-07,2023-05-07 08:58,2023-05-08,748425.01,748425.01,6.357513e+05,4.610057,0.665004,4.695459,4.572559,4.672231,may
1,25,397667.0,UN2501,1085232310,2023-09-02,2023-08-24 19:44,2023-09-06,2801652.00,700413.00,6.357513e+05,4.610057,0.665004,4.695459,4.572559,4.672231,september
2,26,394914.0,UN0375,1085251539,2023-09-11,2023-08-25 02:12,2023-09-13,2475126.00,1237563.00,1.325331e+06,4.610057,0.665004,4.695459,4.572559,4.672231,september
3,29,387370.0,UN2760,1166180157,2024-01-20,2024-01-17 10:00,2024-01-21,1664416.00,1664416.00,1.749181e+06,4.610057,0.665004,4.695459,4.572559,4.672231,january
4,30,398527.0,UN2694,1169990555,2023-08-26,2023-08-24 14:59,2023-08-28,822706.00,411353.00,4.219834e+05,4.610057,0.665004,4.695459,4.572559,4.672231,august


In [87]:
data['contain_national_holiday'] = data.apply(
    lambda row: FE_ContainsHoliday(row['booking_check_in'], row['booking_check_out'], holiday_ranges), axis=1
)

### Create date features

In [88]:
data['booking_check_in'] = pd.to_datetime(data['booking_check_in'])
data['booking_check_out'] = pd.to_datetime(data['booking_check_out'])
data['created_at'] = pd.to_datetime(data['created_at'])

data['stay_duration'] = (data['booking_check_out'] - data['booking_check_in']).dt.days
data['booking_day_of_week'] = data['booking_check_in'].dt.dayofweek
data['booking_lead_time'] = (data['booking_check_in'] - data['created_at']).dt.days


### Calculate revenue metrics

In [89]:
data['total_earnings'] = data['earnings_in_idr']
data['price_fluctuation'] = data['earnings_per_day'] - data['average_baseline_price']

### Calculate review metrics

In [90]:
data['total_review_score'] = (data['rating'] + data['communication'] + data['cleanliness'] + data['accuracy']) / 4

### Display the updated dataframe

In [91]:
data.head()

,Unnamed: 0,room_id,unit_id,booking_id,booking_check_in,created_at,booking_check_out,earnings_in_idr,earnings_per_day,average_baseline_price,...,cleanliness,accuracy,month,contain_national_holiday,stay_duration,booking_day_of_week,booking_lead_time,total_earnings,price_fluctuation,total_review_score
0,13,397667.0,UN2501,07-05-2023MartaRudy,2023-05-07,2023-05-07 08:58:00,2023-05-08,748425.01,748425.01,6.357513e+05,...,4.572559,4.672231,may,False,1,6,-1,748425.01,112673.752803,4.637576
1,25,397667.0,UN2501,1085232310,2023-09-02,2023-08-24 19:44:00,2023-09-06,2801652.00,700413.00,6.357513e+05,...,4.572559,4.672231,september,False,4,5,8,2801652.00,64661.742803,4.637576
2,26,394914.0,UN0375,1085251539,2023-09-11,2023-08-25 02:12:00,2023-09-13,2475126.00,1237563.00,1.325331e+06,...,4.572559,4.672231,september,False,2,0,16,2475126.00,-87767.568155,4.637576
3,29,387370.0,UN2760,1166180157,2024-01-20,2024-01-17 10:00:00,2024-01-21,1664416.00,1664416.00,1.749181e+06,...,4.572559,4.672231,january,False,1,5,2,1664416.00,-84764.976857,4.637576
4,30,398527.0,UN2694,1169990555,2023-08-26,2023-08-24 14:59:00,2023-08-28,822706.00,411353.00,4.219834e+05,...,4.572559,4.672231,august,False,2,5,1,822706.00,-10630.425230,4.637576


In [92]:
filepath=  'FE_RoomPrice.csv'
data.to_csv(filepath)